In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import ElasticNet
from sklearn.metrics import  r2_score
import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb
import xgboost as xgb
from subprocess import check_output
print(check_output(['ls','input']).decode('utf8'))

sample_submission.csv
test.csv
train.csv



/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('input/train.csv', parse_dates=['start_date','creation_date','sell_date'])
test = pd.read_csv('input/test.csv', parse_dates=['start_date','creation_date','sell_date'])

print('Train data size: {} Test data size: {}'.format(train.shape, test.shape))

Train data size: (9366, 18) Test data size: (4801, 17)


In [3]:
train['desk_id'].fillna('DSK00000099', inplace=True)
test['desk_id'].fillna('DSK00000099', inplace=True)

train['desk_id']=train.desk_id.apply(lambda x : x[-4:]).astype(np.int16)
test['desk_id']=test.desk_id.apply(lambda x : x[-4:]).astype(np.int16)

In [4]:
le = LabelEncoder()
le.fit(train['office_id'])
train['office_id'] = le.transform(train['office_id'])
test['office_id'] = le.transform(test['office_id'])

In [5]:
le.fit(train['pf_category'])
train['pf_category'] = le.transform(train['pf_category'])
test['pf_category'] = le.transform(test['pf_category'])

In [6]:
train['start_creation'] = (train.start_date - train.creation_date).dt.days
test['start_creation'] = (test.start_date - test.creation_date).dt.days

In [7]:
train['start_sell'] = (train.start_date - train.sell_date).dt.days
test['start_sell'] = (test.start_date - test.sell_date).dt.days

In [8]:
train['creation_sell'] = (train.creation_date - train.sell_date).dt.days
test['creation_sell'] = (test.creation_date - test.sell_date).dt.days

In [9]:
train['curr_to_eur'] = train['currency'].apply(lambda x : 1.18 if x=='USD' else 1.17  if x=='CHF'
                                               else 133.28 if x=='JPY'  else 1)
test['curr_to_eur'] = test['currency'].apply(lambda x : 1.18 if x=='USD' else 1.17  if x=='CHF' 
                                                else 133.28 if x=='JPY'  else 1)

In [10]:
train['sold'].fillna(68552000.0, inplace= True)
train['bought'].fillna(68427270.0, inplace= True)

In [11]:
train['sold_in_eur'] = train['sold']*train['curr_to_eur']
test['sold_in_eur'] = test['sold']*test['curr_to_eur']

train['bought_in_eur'] = train['bought']*train['curr_to_eur']
test['bought_in_eur'] = test['bought']*test['curr_to_eur']

In [12]:
train['dff_sold_bought'] = (train.sold - train.bought)
test['dff_sold_bought'] = (test.sold - test.bought)

In [13]:
train['gain_loss'] = train.dff_sold_bought * train.euribor_rate *train['curr_to_eur']*train['start_sell']/30
test['gain_loss'] = test.dff_sold_bought * test.euribor_rate *test['curr_to_eur']*test['start_sell']/30

In [14]:
train.drop(['start_date','creation_date', 'sell_date'], inplace=True, axis=1)
test.drop(['start_date','creation_date', 'sell_date'], inplace=True, axis=1)

In [15]:
le.fit(train['country_code'])
train['country_code'] = le.transform(train['country_code'])
test['country_code'] = le.transform(test['country_code'])

In [16]:
le.fit(train['currency'])
train['currency'] = le.transform(train['currency'])
test['currency'] = le.transform(test['currency'])

In [17]:
train['libor_rate'].fillna(0.0, inplace= True)
test['libor_rate'].fillna(0.0, inplace=True)

In [18]:
train['sold_libor'] = train['sold']*train['libor_rate']
test['sold_libor'] = test['sold']*test['libor_rate']

train['sold_euribor'] = train['sold']*train['euribor_rate']
test['sold_euribor'] = test['sold']*test['euribor_rate']

In [19]:
train['indicator_code'].fillna(False, inplace= True)
test['indicator_code'].fillna(False, inplace= True)

le.fit(train['indicator_code'])
train['indicator_code'] = le.transform(train['indicator_code'])
test['indicator_code'] = le.transform(test['indicator_code'])

In [20]:
le.fit(train['type'])
train['type'] = le.transform(train['type'])
test['type'] = le.transform(test['type'])

In [21]:
train['hedge_value'].fillna(False, inplace=True)
test['hedge_value'].fillna(False, inplace=True)

train['hedge_value'] = train['hedge_value'].map({False:0, True:1})
test['hedge_value'] = test['hedge_value'].map({False:0, True:1})

In [22]:
train['status'].fillna(False, inplace= True)
test['status'].fillna(False, inplace= True)

train['status'] = train['status'].map({False:0, True:1})
train['status'] = train['status'].map({False:0, True:1})

In [23]:
retn = train['return']
test_portfolio_id = test.portfolio_id.values

#train.drop(['portfolio_id','bought', 'type', 'return'], inplace=True, axis=1)
#test.drop(['portfolio_id','bought', 'type'], inplace=True, axis=1)
train.drop(['portfolio_id',  'return'], inplace=True, axis=1)
test.drop(['portfolio_id'], inplace=True, axis=1)

In [140]:
params = {'learning_rate': 0.04, 
          'num_leaves':70,
          'min_data_in_leaf': 120,
          'max_depth': -1, 
          'colsample_bytree': 0.8,#0.522,
          'boosting': 'gbdt', 
          'application': 'regression', 
          "min_child_samples": 10,
          'metric': 'mae',  
          'verbosity': 0,
          'seed': 32}

In [141]:
lgb_train = lgb.Dataset(train, label=np.log1p(retn))

In [142]:
lgb_cv = lgb.cv(params, lgb_train, num_boost_round=10000, nfold= 5, stratified=False,
                 verbose_eval=100, early_stopping_rounds=40)

[100]	cv_agg's l1: 0.00152072 + 7.49139e-05
[200]	cv_agg's l1: 0.00131357 + 8.49467e-05
[300]	cv_agg's l1: 0.00123613 + 7.29904e-05
[400]	cv_agg's l1: 0.00119929 + 6.43389e-05
[500]	cv_agg's l1: 0.00118254 + 5.91141e-05
[600]	cv_agg's l1: 0.00117745 + 5.3454e-05


In [143]:
nround = lgb_cv['l1-mean'].index(np.min(lgb_cv['l1-mean'])) #mae
#nround = lgb_cv['rmse-mean'].index(np.min(lgb_cv['rmse-mean'])) #rmse
print(nround)

622


In [144]:
model = lgb.train(params, lgb_train, num_boost_round=nround)

In [145]:
r2_score(retn, np.expm1(model.predict(train)))

0.86386294454307877

In [95]:
#lgb.plot_importance(model)

In [96]:
pred = model.predict(test)

In [97]:
sub = pd.DataFrame({'portfolio_id':test_portfolio_id, 'return':np.expm1(pred)})
sub.head(5)

,portfolio_id,return
0,PF00001001,0.028634
1,PF00001004,0.037468
2,PF00001009,0.023994
3,PF00001013,0.025118
4,PF00001014,0.024616


In [98]:
sub.to_csv('lgb_v4.csv', index=False)

In [104]:
en = ElasticNet(max_iter=10000, normalize=True)
lr = en.fit(train, retn)

In [105]:
#r2_score(retn, lr.predict(train))
lr.predict(train)

array([ 0.01409979,  0.01409979,  0.01409979, ...,  0.01409979,
        0.01409979,  0.01409979])

In [108]:
def xgb_r2(pred, drain):
    y = list(drain.get_label())
    score = r2_score(y, np.expm1(pred))
    return 'r2_score', score
    

In [115]:
xgb_params = {
    'seed': 4,
    'colsample_bytree': 0.73,
    'silent': 1,
    'subsample': 0.63,
    'learning_rate': 0.04,
    'objective':'reg:linear',    
    #'max_depth': 6,
    'num_parallel_tree':8 ,
    'min_child_weight': 2,
    #'gamma': 0.046,
    #'reg_alpha': 1.39,    
}

In [116]:
dtrain = xgb.DMatrix(data=train, label=np.log1p(retn))
dtest = xgb.DMatrix(data=test)

In [117]:
res = xgb.cv(xgb_params, dtrain, num_boost_round=2000, nfold=5,  stratified=False, feval = xgb_r2,
             early_stopping_rounds=25, verbose_eval=100, show_stdv=True,  maximize=True)


[0]	train-r2_score:-1057.67+45.5895	test-r2_score:-1079.6+145.016
[20]	train-r2_score:-644.036+27.7566	test-r2_score:-657.411+88.4089
[40]	train-r2_score:-399.232+17.2032	test-r2_score:-407.539+54.8797
[60]	train-r2_score:-250.994+10.8135	test-r2_score:-256.227+34.553
[80]	train-r2_score:-159.547+6.87373	test-r2_score:-162.885+22.0019
[100]	train-r2_score:-102.278+4.40611	test-r2_score:-104.426+14.1321
[120]	train-r2_score:-65.97+2.84268	test-r2_score:-67.3637+9.13765
[140]	train-r2_score:-42.7175+1.84184	test-r2_score:-43.6285+5.93569
[160]	train-r2_score:-27.7014+1.19619	test-r2_score:-28.302+3.86508
[180]	train-r2_score:-17.9384+0.777338	test-r2_score:-18.3382+2.51575
[200]	train-r2_score:-11.5538+0.504094	test-r2_score:-11.8236+1.6318
[220]	train-r2_score:-7.35787+0.325502	test-r2_score:-7.54458+1.04937
[240]	train-r2_score:-4.58897+0.208218	test-r2_score:-4.72309+0.664042
[260]	train-r2_score:-2.75489+0.131378	test-r2_score:-2.85628+0.408556
[280]	train-r2_score:-1.53601+0.0809016

In [118]:
best_nrounds = res.shape[0] - 1
cv_mean = res.iloc[-1, 0]
cv_std = res.iloc[-1, 1]

print('xgb-CV: {0}+{1}'.format(cv_mean, cv_std))

xgb-CV: 0.8499464+0.1370017069639645


In [119]:
gbdt = xgb.train(xgb_params, dtrain, best_nrounds)

In [120]:
pred_1 = gbdt.predict(dtest)

In [121]:
sub1 = pd.DataFrame({'portfolio_id':test_portfolio_id, 'return':np.expm1(pred_1)})
sub1.head(5)

,portfolio_id,return
0,PF00001001,0.025977
1,PF00001004,0.034628
2,PF00001009,0.024494
3,PF00001013,0.025515
4,PF00001014,0.024695


In [122]:
sub1.to_csv('xgb_v2.csv', index=False)

In [490]:
pred2 = np.expm1(0.6*pred + 0.4*pred_1)

In [492]:
sub2 = pd.DataFrame({'portfolio_id':test_portfolio_id, 'return':pred2})
sub2.head(5)

,portfolio_id,return
0,PF00001001,0.027049
1,PF00001004,0.033083
2,PF00001009,0.024576
3,PF00001013,0.026555
4,PF00001014,0.024326


In [493]:
sub2.to_csv('lgb_xgb_v1.csv', index=False)